In [40]:
#!/usr/bin/env python
from urllib.request import urlopen
import sys

# This script takes in the uniprot information and search on the European Nucleotide Archive (ENA) and fetch the nucleotide sequences in Fasta.

# Read the uniprot.tab and uniprot.txt
#Utab=open("uniprot_reviewed.tab","r")
Utxt=open("uniprot-reviewed.txt","r")

accessionList=[]# This is the list that contains all the CDS accession number in ENA
for line in Utxt:
    index=line.find('CP001396')
    if  index != -1:
        accessionList.append(line[index+10:index+20])
        #print(line[index+10:index+20])

#print(len(accessionList))

In [41]:
fasta = open('candidates.fa', 'w')

# accquire the fasta file from ENA by the accession List
for idx,accession in enumerate(accessionList):
    link = "https://www.ebi.ac.uk/ena/data/view/{}&display=fasta".format(accessionList[idx])
    single_f =urlopen(link).read().decode("utf-8")
    fasta.write(single_f)

fasta.close()
# fasta = open('candidates.fa', 'r')
# print(fasta.read())

In [90]:
# This is basically a fasta reader and we add [] flanking the sequences to specify target product
class FASTAReader( object ):
    def __init__( self, file ):
        self.file = file
        self.last_id = None
        
    def next( self ):
        if self.last_id is None:
            line = self.file.readline()
            # Verify is header line
            assert line.startswith( ">" )
            # Extract id -- whole line
            ## identifier = line[1:].rstrip( "\r\n" )
            # Extract id -- space
            identifier = line[0:].split()[0]
        else:
            identifier = self.last_id

        sequences = []

        while True:
            line = self.file.readline().rstrip("\r\n")
            if line.startswith( ">" ):
                self.last_id = line[0:].split()[0]
                break
            elif line == "":
                sequences.append( line )
                return False, identifier, "".join( sequences )
            else:
                sequences.append( line )
        #sequences.append("]")
        return True, identifier, "".join( sequences )

# This is to generate the specified product for batchPrimer3 design
target = open('candidates_target.fa','w')
fasta = open('candidates.fa', 'r')

reader = FASTAReader(fasta)

while True:
    mybool, identifier, sequence=reader.next()
    target.write(identifier)
    target.write("\n")
    #target.write("[")
    target.write(sequence[0:-3])# get rid of stop codon
    #target.write("]")
    target.write("\n")
    if mybool is False:
        break
target.close()
fasta.close()

In [ ]:

#     line = myfile.readline()
#     # Verify is header line
#     assert line.startswith( ">" )
#     # Extract id -- whole line
#     ## identifier = line[1:].rstrip( "\r\n" )
#     # Extract id -- space
#     identifier = line[1:].split()[0]


#aa_codes=Utxt.readline().split()
# for line in f:
#     values=line.split()
#     row=values[0]
#     values=values[1:]
#     for index in range(0,len(aa_codes)):
#         blosum50[row,aa_codes[index]]=float(values[index])
# f.close()






# f = open('seqs.fasta','a')
# for i in range(0,100):
#     f.write(">RandomMutationSeq_{}\n".format(i+1))
#     for r in sequence:
#         f.write(mutate(r))
#     f.write("\n")
    
# f.close()